In [ ]:
import os
print(os.getcwd())


: 

In [3]:
# === STEP 1: Imports ===
import pickle
import chromadb

# === STEP 2: Load embeddings and metadata ===
with open("../data/others/projects_embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

with open("../data/others/projects_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

import json

for m in metadata:
    json.dumps(m)   # ensure serializable; raises error if not
    
# === STEP 3: Initialize Chroma (new API, no deprecated Settings) ===
client = chromadb.PersistentClient(path="data/chroma_db")

# === STEP 4: Create or get collection ===
if "mscac_projects" in [c.name for c in client.list_collections()]:
    collection = client.get_collection("mscac_projects")
    print("mscac projects already exists")
else:
    collection = client.create_collection(
        name="mscac_projects",
        metadata={"description": "MScAC alumni internship projects"}
    )



Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


mscac projects already exists


In [4]:
# === STEP 5: Add embeddings and metadata ===
import numpy as np

embeddings = [e.tolist() if isinstance(e, np.ndarray) else e for e in embeddings]
collection.add(
    embeddings=embeddings,                 # list/array of embeddings
    documents=[m.get("internship_title_(aria)", "") for m in metadata],  # short text for display
    metadatas=metadata,                    # full metadata dicts
    ids = [str(m["project_id"]) for m in metadata])


Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


In [11]:
# === STEP 6: Test a query ===
# For now, just use the first embedding as a test
query_embedding = embeddings[0]

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

print(results)

{'ids': [['1261', '1016', '618']], 'embeddings': None, 'documents': [['AI Compiler Methods to Improve GPU Runtime Performance', 'Predicting Register Requirements for Program Optimization', 'Machine learning for pattern-matching optimization in GPU compilers']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'concentration': 'Artificial Intelligence', 'partner_supervisor': 'Andrei Yershov', 'internship_title_(aria)': 'AI Compiler Methods to Improve GPU Runtime Performance', 'project_id': 1261, 'academic_supervisor': 'Angela Demke Brown', 'cohort': '2024–25', 'partner': 'AMD', 'student': 'Eric Zheng'}, {'partner': 'AMD', 'partner_supervisor': 'Olga Golovanevsky', 'internship_title_(aria)': 'Predicting Register Requirements for Program Optimization', 'academic_supervisor': 'Nandita Vijaykumar', 'cohort': '2023–24', 'student': 'Govind Sudarshan', 'project_id': 1016, 'concentration': 'Computer Science'}, {'partner': 'AMD', 'academic_supervis